In [1]:
import pandas as pd
import numpy as np
import ast
import seaborn as sns
import gensim
from gensim.models import Word2Vec
import torch
from models import LyricsGenerator, LyricsDataset, MergeLyricsGenerator
from tools import generate_text


# Evaluation
## 1. experiment reuslts proccseing

In [2]:
df_results=pd.read_csv('results/summary_results.csv')
df_results["validation_perplexities"]=df_results["validation_perplexities"].apply(ast.literal_eval)
df_results["train_perplexities"]=df_results["train_perplexities"].apply(ast.literal_eval)

df_results["min_perplexity_val"]=df_results["validation_perplexities"].apply(min)
df_results["min_perplexity_train"]=df_results["train_perplexities"].apply(min)
df_results['min loss epoch'] = df_results['validation_perplexities'].apply(lambda x: x.index(min(x)))


In [3]:
df_results.sort_values(by='min_perplexity_val',inplace=True)
df_results.to_csv("results/df_results_processed.csv")

## 2. Choose the best model
1. {epoch : 15, hidden dimensions : 40, lstm layer : 2, batch size: 16, sequence lenght: 5, learning rate : 0.0001,dropout:0.1, model:  merge, midi embedding: graph} <br>
2. {epoch : 7, hidden dimensions : 40, lstm layer : 2, batch size: 16, sequence lenght: 5, learning rate : 0.001
,dropout:0.1, model:  naive, midi embedding: modified}

In [4]:
test_df=pd.read_csv('lyrics_test_set.csv')
midi_graph_embedding=pd.read_csv('matched_embeddings_graph.csv')
midi_modfieid_embedding=pd.read_csv('matched_embeddings_modified.csv')
test_df_grpah=test_df.merge(midi_graph_embedding,on=['singer',	'song'])
test_df_modified=test_df.merge(midi_modfieid_embedding,on=['singer',	'song'])

### 2.1 model 1: Merge Model with graph embedding

merge model with graph embedding

In [5]:
#import word2vec moudle 
word2vec_moudle = Word2Vec.load("results/word2vec_graph.model")
word_to_idx = {word: idx for idx, word in enumerate(word2vec_moudle.wv.index_to_key)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
weights = torch.FloatTensor(word2vec_moudle.wv.vectors)
vocab_size=len(word_to_idx)

#import models
best_model_graph = MergeLyricsGenerator(vocab_size, embedding_dim=300, hidden_dim=40, num_layers=2,dropout=0.1)
best_model_graph.load_state_dict(torch.load('results/best_model_graph.pth'))
best_model_graph.eval()  # Set the model to evaluation mode


MergeLyricsGenerator(
  (embedding): Embedding(8373, 300)
  (midi_dense): Linear(in_features=50, out_features=5, bias=True)
  (relu): ReLU()
  (lstm): LSTM(305, 40, num_layers=2, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=40, out_features=8373, bias=True)
)

In [25]:
test_df_grpah

,singer,song,lyrics,1,2,3,4,5,6,7,...,41,42,43,44,45,46,47,48,49,50
0,billy joel,honesty,if you search for tenderness & it isn't hard ...,-0.765890,0.296827,-0.655231,-0.282960,-0.085955,-0.861380,0.638619,...,0.903643,-0.255035,-0.280794,-0.538465,1.032473,0.462760,-0.571529,0.095100,1.007492,0.492081
1,cardigans,lovefool,dear i fear we're facing a problem & you love...,-0.792777,1.016099,-0.437489,-0.450564,-0.434409,0.139522,0.091345,...,0.447946,-0.813263,0.632883,-0.188807,-0.413894,0.577169,-0.300781,-0.400971,-0.271699,0.319355
2,aqua,barbie girl,hi ya barbie & hi ken! & do you want to go fo...,-0.001985,0.052934,0.042573,-0.513248,-1.163136,-0.374324,0.381804,...,1.267293,0.233147,0.238585,-0.204469,-0.070003,0.811581,1.361675,0.301339,0.027747,-0.613182
3,blink 182,all the small things,all the small things & true care truth brings...,-0.704593,0.568510,-0.310890,-1.013620,-0.624437,-1.196120,0.046716,...,0.678384,-0.260534,-0.160154,-0.607179,0.108219,0.091889,-0.570823,-0.213483,-0.438360,-0.892683


In [24]:
for index, row in test_df_grpah.iterrows():
    midi_embedding=row.iloc[3:]
    first_word=row['lyrics'].split(' ')[1]
    sequence_length=5
    max_length=100
    
    new_song=generate_text(first_word, best_model_graph,sequence_length, max_length, vocab_size, word_to_idx, idx_to_word, word2vec_moudle, midi_embedding) 
    print(new_song)    

if for you & again when [chorus: & & don't ain't but comin' & & & i'll true girls love looking & & in your heart & and you have i could just & i'll just to go & & i have & to real value & & baby checkin' for learn & & love what someday & & by you can said to cause & & gone would & i sorry sandman music & it's just & to right to go you're here just will just to go & & i've go & by is me love never well
dear we come & there i could see & & i how will live watch your go and i'm better christmas & & tv-funk in your city & to heal amadeus there so show so remember & & will just takes blue daddy! loving i it's much them one doin' & & that even & i icy up could spell devil in side & safe square beside anything & love maybe & to start & & all i those peacefully & & of listen & dawn she soulmates & love stand & like you could i &
hi you love & & around & like bring by my without i taught to & & won't mistake he nightmare & & he used & & & & & whenever me & in happy he's and it taken & & & all &

In [34]:
for first_word in ["sunny",'life','red']:
    
    for index, row in test_df_grpah.iterrows():
        midi_embedding=row.iloc[3:]
        sequence_length=5
        max_length=100
        
        new_song=generate_text(first_word, best_model_graph,sequence_length, max_length, vocab_size, word_to_idx, idx_to_word, word2vec_moudle, midi_embedding) 
        print(new_song)   

sunny desperado & & & & i've lick & and something love love & & live and would you & for a closer & sweet broken door i'm on and sparks they give & to put on free be arms and good to kiss & by the years we'll he's da & & up hurts you song what you're can't it come & well & & of my cold began & babies & & papers i eyes have & & i go & & killing out some all or i'm &
sunny & this only only i've is well that look to be on & the save & at we in regular & & i yea love life & to time that listen again & & & & the miss have & & can she tonight and come are my that's little oh oh oh oh oh oh oh oh shit & to say & & & in my heart man & past i will said oh i you're do & i have & to can't think & he want & baby & so used
sunny about & & & oh oh oh oh oh fine yeah & & slap & & & & i need to in you & & walk to draw and i much you love & & & & walked in your just for the fire & & have & for man & to keep you happy they taught & & for it true no & like view & & & & & & & i didn't south & her meet & &

### 2.2 model 2: Naive mode + modified embedding


In [26]:
import torch.nn as nn


In [27]:
#import word2vec moudle 
word2vec_moudle = Word2Vec.load("results/word2vec.model")
word_to_idx = {word: idx for idx, word in enumerate(word2vec_moudle.wv.index_to_key)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}
weights = torch.FloatTensor(word2vec_moudle.wv.vectors)
vocab_size=len(word_to_idx)
#import models
best_model_modified = LyricsGenerator(vocab_size, embedding_dim=350, hidden_dim=40, num_layers=2,dropout=0.1)
best_model_modified.embedding.weight = nn.Parameter(weights)

best_model_modified.load_state_dict(torch.load('results/best_model_modified.pth'),strict=False)
best_model_modified.eval()  # Set the model to evaluation mode


LyricsGenerator(
  (embedding): Embedding(8310, 350)
  (lstm): LSTM(350, 40, num_layers=2, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=40, out_features=8310, bias=True)
)

In [28]:
for index, row in test_df_grpah.iterrows():
    midi_embedding=row.iloc[3:]
    first_word=row['lyrics'].split(' ')[1]
    sequence_length=5
    max_length=100
    
    new_song=generate_text(first_word, best_model_graph,sequence_length, max_length, vocab_size, word_to_idx, idx_to_word, word2vec_moudle, midi_embedding) 
    print(new_song)    

if & i'm the waitin' & & & love to be me. & & i've cryin' i attitude crack & i have & in along- & who ought & & don't slow in this shit do & & love i come do and says way & & & & love you we'll be this to songs your way again? you to are & the shot & & love & in my own & said their fantasic & of me & & you would it can't la & & & that i'm only place at me
dear & & i can in. & & we'll hand from the night love love & i have & & & & love i'll be hot & i thing & for me & & you can right & & like your heard? & love opened & i have & & & come & and you could i name now & & my vain only when let gimme & do nowhere & & i much just and i will you'll are wanna & & only way i have like & & around & &
hi a leave i & & don't knows it yeah & & love & like you to time i say believe & & it i come & & i'm love & i'm thing & loved & i just can't go & & ever & i can blame? & from years flex & & & the world love soul remember & & in your voice his time & & the is left it for a country be-e-ell & & who wil